In [ ]:
import os, sys, time, random, itertools
from datetime import datetime

from google.colab import drive
drive.mount('/content/drive')

mygdrive = '/content/drive/MyDrive/ART/design+computationalthinking/code/'
sys.path.append(mygdrive)
from pyt_utilities import *

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# variables
#root = '/home/marcbohlen/'
root = '/content/drive/MyDrive/ART/design+computationalthinking/'
path = root + 'data/'
dataset = 'bali10'
#----------------------------------------
#pick a convolutional neural network model
#vanillanet, alexnet, squeezenet, resnet18, resnet152, resnext50
network = 'resnet152'
#----------------------------------------
pretrained = False;
balinorms = True;           #normalization based on the bali-26 collection. See create_norms.py

num_epochs = 20;
offset = 0; img_limit = 1000; randomprune = True;
training_percentage = 0.8;
tpp = int(100 * training_percentage)

In [ ]:
datapath = path + dataset + '/'
resultspath = root + 'results/'
if not os.path.exists(resultspath):
    os.makedirs(resultspath)

In [ ]:
if(pretrained == True):
    if(balinorms == True):
        checkpointname = path + dataset + '_' + network + '_pretrainedyes_t' + str(tpp) + '_e' + str(num_epochs) + '_lim' + str(img_limit) + '_balinorms'+ '_checkpoint.pth'
        output = resultspath + dataset + '_' + network + '_pretrainedyes_t' + str(tpp) + '_e' + str(num_epochs) + '_lim' + str(img_limit) + '_balinorms'+ '.jpg'
        comment_t = dataset + ', ' + network + ', pretrainedyes_t' + str(tpp) + ', e' + str(num_epochs) + ', lim' + str(img_limit) + '_balinorms'
        filename = resultspath + dataset + '_' + network + '_pretrainedyes_t' + str(tpp) + '_e' + str(num_epochs) + '_lim' + str(img_limit) + '_balinorms'+ '.csv'
    else:
        checkpointname = path + dataset + '_' + network + '_pretrainedyes_t' + str(tpp) + '_e' + str(num_epochs) + '_lim' + str(img_limit) + '_checkpoint.pth'
        output = resultspath + dataset + '_' + network + '_pretrainedyes_t' + str(tpp) + '_e' + str(num_epochs) + '_lim' + str(img_limit) + '.jpg'
        comment_t = dataset + ', ' + network + ', pretrainedyes_t' + str(tpp) + ', e' + str(num_epochs) + ', lim' + str(img_limit)
        filename = resultspath + dataset + '_' + network + '_pretrainedyes_t' + str(tpp) + '_e' + str(num_epochs) + '_lim' + str(img_limit) + '.csv'
else:
    if(balinorms == True):
        checkpointname = path + dataset + '_' + network + '_pretrainedNO_t' + str(tpp) + '_e' + str(num_epochs) + '_lim' + str(img_limit) + '_balinorms'+ '_checkpoint.pth'
        output = resultspath + dataset + '_' + network + '_pretrainedyNO_t' + str(tpp) + '_e' + str(num_epochs) + '_lim' + str(img_limit) + '_balinorms'+ '.jpg'
        comment_t = dataset + ', ' + network + ', pretrainedNO_t' + str(tpp) + ', e' + str(num_epochs) + ', lim' + str(img_limit) + '_balinorms'
        filename = resultspath + dataset + '_' + network + '_pretrainedNO_t' + str(tpp) + '_e' + str(num_epochs) + '_lim' + str(img_limit) + '_balinorms' + '.csv'
    else:
        checkpointname = path + dataset + '_' + network + '_pretrainedNO_t' + str(tpp) + '_e' + str(num_epochs) +'_lim' + str(img_limit) +  '_checkpoint.pth'
        output = resultspath + dataset + '_' + network + '_pretrainedNO_t' + str(tpp) + '_e' + str(num_epochs) +'_lim' + str(img_limit) +  '.jpg'
        comment_t = dataset + ', ' + network + ', pretrainedNO_t' + str(tpp) + ', e' + str(num_epochs) + ', lim' + str(img_limit)
        filename = resultspath + dataset + '_' + network + '_pretrainedNO_t' + str(tpp) + '_e' + str(num_epochs) + '_lim' + str(img_limit) + '.csv'


In [ ]:
image_datasets = {x: datasets.ImageFolder(os.path.join(datapath, x), data_transforms[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
device = ("cuda" if torch.cuda.is_available() else "cpu" )


In [ ]:
print('checkpoint at: ', checkpointname)

checkpoint at:  /content/drive/MyDrive/ART/design+computationalthinking/data/bali10_resnet152_pretrainedNO_t80_e20_lim1000_balinorms_checkpoint.pth


In [ ]:

try:
    model = load_checkpoint(checkpointname)
except:
    print('That model does not exist... ')
    exit()

print('\n> Predicting class of input images <')

for i in range (0, len(class_names)):
    plantdatapath = datapath + 'val/' + class_names[i] + '/'
    path, dirs, files = next(os.walk(plantdatapath))
    limit = len(files)
    top1 = 0; topN = 0; tk = 3

    for j in range (0, limit):
        image_path = next(itertools.islice(os.scandir(plantdatapath), j, None)).path
        predictions, percentage, outcategory = predict_image(image_path, model, predict_transform, class_names, tk)
        topN_ind = predictions[1].tolist()[0]
        top1_ind = topN_ind[0]

        input = image_path.split('/')[-2], image_path.split('/')[-1]

        #you can check input[1] to find problematic images / categories of confusion
        #print('\ninput: ', input); print('output: ', outcategory, percentage)

        if(class_names[top1_ind] == input[0]):
            top1 = top1 + 1

        if(check_topN(class_names, topN_ind, tk, input[0]) == 1):
            topN = topN + 1

    top1_score = float(top1 / limit)
    topN_score = float(topN / limit)
    top1_error = 100*float("%.3f" %(1.0 - top1_score))
    topN_error = 100*float("%.3f" %(1.0 - topN_score))

    comment = comment_t + ', ' + str(class_names[i])  + ', top1-error, ' + str(top1_error) + ', top' + str(tk) + '-error, ' + str(topN_error)
    print(comment + '\n')
    write2file(filename, comment)

got this far into loading checkpoint...
now got this far into loading checkpoint...

> Predicting class of input images <
bali10, resnet152, pretrainedNO_t80, e20, lim1000_balinorms, aroid, top1-error, 56.00000000000001, top3-error, 32.5

bali10, resnet152, pretrainedNO_t80, e20, lim1000_balinorms, dragonfruit, top1-error, 47.0, top3-error, 19.5

bali10, resnet152, pretrainedNO_t80, e20, lim1000_balinorms, frangipani, top1-error, 73.5, top3-error, 46.0

bali10, resnet152, pretrainedNO_t80, e20, lim1000_balinorms, jackfruit, top1-error, 34.0, top3-error, 17.0

bali10, resnet152, pretrainedNO_t80, e20, lim1000_balinorms, nilam, top1-error, 18.0, top3-error, 1.5

bali10, resnet152, pretrainedNO_t80, e20, lim1000_balinorms, papaya, top1-error, 76.5, top3-error, 22.0

bali10, resnet152, pretrainedNO_t80, e20, lim1000_balinorms, passiflora, top1-error, 57.99999999999999, top3-error, 27.500000000000004

bali10, resnet152, pretrainedNO_t80, e20, lim1000_balinorms, sawo, top1-error, 65.0, top3-